# Hyperparameter Tuning using HyperDrive


In [1]:

# import dependencies
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset

from azureml.core import ScriptRunConfig

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.19.0


In [10]:
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="capstone-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-131753
Azure region: southcentralus
Subscription id: 9b72f9e6-56c5-4c16-991b-19c652994860
Resource group: aml-quickstarts-131753


## Dataset

In [3]:
# NOTE: update the key to match the dataset name
found = False
key = "diabetes-classification"
description_text = "Diabetes datasets for capstone project"

if key in ws.datasets.keys(): 
        found = True        
        dataset = ws.datasets[key] 

if not found:
        # Create Hyperdrive Dataset and register it into Workspace
        url = 'https://raw.githubusercontent.com/purunep/Capstone/main/project/data/diabetes.csv'
        dataset = Dataset.Tabular.from_delimited_files(url)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

from train import clean_data
[x_data, y_data] = clean_data(dataset)

In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

cpu_cluster_name = "puru-compute"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D12_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


In [5]:
from azureml.widgets import RunDetails
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os
import joblib

In [41]:
env = Environment('puru-environment')
print(env)

Environment(Name: puru-environment,
Version: None)


In [8]:
!pip install -U scikit-learn

     |████████████████████████████████| 22.2 MB 5.3 MB/s eta 0:00:01
ERROR: azureml-train-automl-runtime 1.19.0 has requirement scikit-learn<0.23.0,>=0.19.0, but you'll have scikit-learn 0.24.0 which is incompatible.
ERROR: azureml-automl-runtime 1.19.0 has requirement scikit-learn<0.23.0,>=0.19.0, but you'll have scikit-learn 0.24.0 which is incompatible.
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [12]:
from azureml.core import Environment
envit = Environment.get(workspace=ws,name='AzureML-Tutorial')

In [ ]:
## Hyperdrive Configuration

In [14]:

# Specify parameter sampler
ps = RandomParameterSampling( {
        "--C": uniform(0.50,1.00),
       "--max_iter" : choice(10,20,30)
    }
)

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)



#sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='./conda_dependencies.yml')
#envit = Environment.get(workspace=ws,name='myenv')

config  = ScriptRunConfig(source_directory='./',
                         script='train.py',                         
                         compute_target=cpu_cluster,
                         environment=envit)



#hyperdrive_run_config = <your config here>
# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
                                   hyperparameter_sampling = ps,
                                   primary_metric_name = 'Accuracy',
                                   max_total_runs = 50,
                                   max_concurrent_runs = 4,
                                   primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                   policy = policy,
                                   run_config = config)

#estimator=estimator use to replace estimator in hyperdriveconfig

In [15]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()
# wait for completion
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [20]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()


print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])

Best Run Id:  HD_62a3d528-63c1-4675-b21a-f5969d4d0215_3

 Accuracy: 0.7272727272727273


In [21]:
# register model for future deployment
# os.makedirs('outputs', exist_ok=True)

description = 'Diabetec'
tags = {'area': 'health', 'type': 'classification'}

hyperdrive_model = best_run.register_model(model_name='puru-model',
                                                      model_path='.',
                                                      tags=tags,
                                                      description=description)

print(hyperdrive_model.name, hyperdrive_model.id, hyperdrive_model.version, sep='\t')

puru-model	puru-model:1	1


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service